# Concatenating, Merging, and Compare

Pandas provides various facilities for easily combining together Series or DataFrame with various kinds of set logic.

In addition, pandas also provides utilities to compare two Series or DataFrame and summarize their differences.

Topic Reviewed

- Concatenation (vertical and horizontal) `concat()`
- Merging `merge()` and `join()`
- Comparison `compare()`

In [28]:
import pandas as pd
import numpy as np

np.random.seed(0)

In [56]:
# Handy Functions
from IPython.display import display_html

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def letter_list(letter, length, start = None):
    if start:
        result = [f"{letter}{i}" for i in range(start, length + start)]
    else:
        result = [f"{letter}{i}" for i in range(length)]
    return result


## Concatenation: `concat()`

The function `concat()` is used to concat two (or more) DataFrames or Series
along an axis, and additionaly perform optional set logic.

`concat(objs, axis, join, ignore_index, keys)`

- `objs`: list of objects (`DataFrame` or `Series`) or dict of objects
- `axis`: the axis to concatenate along (default 0)
- `join`: how to handle repeated indexes with different information
    - `join = outer` performs an union (default and zero information loss)
    - `join = inner` perform an interception
- `ignore_index` (default False):  If True, do not use the index values **on the concatenation axis**. The resulting axis will be the default. Note the index values on the other axes are still respected in the join.
- `keys`: construct hierarchical index using the passed keys as the outermost level. It is useful to indentidy the source of each chunk of data in the resulting `DataFrame` (or `Series`)

**NOTE**: Take in mind that `concat()` **makes a full copy of the data**, and that constantly reusing this function can create a significant performance hit. If you need to use the operation over several datasets, use a list comprehension.

Things to take into account about `concat()`

1. Vertical and Horizontal Stacking
2. `keys` and `ignoring_index`
3. Difference between `join = outer` and `join = inner`
4. Concatenating with mixed dimensions

### Vertical and Horizontal Stacking 

To control the way to stack, you can use `axis` parameter

- Vertical Stacking with `axis = 0` (default)
- Horizontal Stacking with `axis = 1` (default)

**NOTE**: if there are repeated labels along the axis. All the labels in that axis will appear on the result although the result will have duplicate labels in that axis.

<img src="./assets/imgs/concat_h_v_stacking.png" width="600"/>

In [30]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[0, 1, 2, 3],
)
display_side_by_side(df1, df2)


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
,A,B,C,D
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


In [31]:
# 1. By default concat along the axis = 0
# NOTE: Although there are duplicate index labels, they are kept.
pd.concat([df1, df2])

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


In [32]:
# 2. Concat along the axis = 1
# NOTE: Although there are duplicate column (axis = 1) labels, they are kept.
pd.concat([df1, df2], axis=1)

,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,A4,B4,C4,D4
1,A1,B1,C1,D1,A5,B5,C5,D5
2,A2,B2,C2,D2,A6,B6,C6,D6
3,A3,B3,C3,D3,A7,B7,C7,D7


### `keys` (or dict) and `ignoring_index`

There are two ways to address duplicated labels:

1. The `keys = <list of identifiers>` can be used to create a multi-index that identifies each group (chunk of joined data) in the resulting `DataFrame` (or `Series`).
2. The `ignoring_index = True` instead will reset and drop the labels, and place the default labels instead `0, 1, ...`. It is commonly used when you have object with no meaningful labels.

**NOTE:** You can get the same result of `keys` using a dictionary as input to `objs` (first parameter).

**NOTE:** `keys` and `ignorind_index` will hava effect on `index` or `columns` depending what the `axis` indicates.

In [33]:
# 1. Construct multi-index to identify chunks of data using keys
# NOTE: you can index result.loc["df1"] to get chunk of data
previous = pd.concat([df1, df2])
result = pd.concat([df1, df2], keys=["df1", "df2","df3"])
display_side_by_side(previous, result)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 0 
 A4 
 B4 
 C4 
 D4 
 
 
 1 
 A5 
 B5 
 C5 
 D5 
 
 
 2 
 A6 
 B6 
 C6 
 D6 
 
 
 3 
 A7 
 B7 
 C7 
 D7 
 
 
 
 
 
 
 
 A 
 B 
 C 
 D 
 
 
 
 
 df1 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 df2 
 0 
 A4 
 B4 
 C4 
 D4 
 
 
 1 
 A5 
 B5 
 C5 
 D5 
 
 
 2 
 A6 
 B6 
 C6 
 D6 
 
 
 3 
 A7 
 B7 
 C7 
 D7

In [34]:
#NOTE: you can also use a dict to get the same behavior
pd.concat({"df1": df1, "df2": df2})

A   B   C   D
df1 0  A0  B0  C0  D0
    1  A1  B1  C1  D1
    2  A2  B2  C2  D2
    3  A3  B3  C3  D3
df2 0  A4  B4  C4  D4
    1  A5  B5  C5  D5
    2  A6  B6  C6  D6
    3  A7  B7  C7  D7

In [35]:
# 2. Ignoring index will reset and drop labels
previous = pd.concat([df1, df2])
result = pd.concat([df1, df2], ignore_index=True)
display_side_by_side(previous, result)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7
,A,B,C,D
0,A0,B0,C0,D0


###  `join='outer'` and `join='inner'` difference

`join` attribute controls how to handle **the other axis** (other than the one being concatenated).

- `join = outer` (default) will take the **union** of the other axis. It has zero information loss.
- `join = inner` will take the **interception** of the other axis.

**NOTE:** the normal vertical or horizonal stacking will happen in the main axis (using the `axis` attribute), while the `join` will only control `union` or `interception` in the other axis. e.g: if we define the main `axis = 0` then the `other axis = 1`.

**NOTE:** if some values are missing when use `join = outer` (union), they will be replaced by NaN.

<img src="./assets/imgs/concat_join_parameter.png" width="600"/>

In [36]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

df2 = pd.DataFrame(
    {
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[2, 3, 4, 5],
)

display_side_by_side(df1, df2)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 
 
 
 
 B 
 C 
 D 
 
 
 
 
 2 
 B4 
 C4 
 D4 
 
 
 3 
 B5 
 C5 
 D5 
 
 
 4 
 B6 
 C6 
 D6 
 
 
 5 
 B7 
 C7 
 D7

In [37]:
# vertical stacking along axis = 0
# and join = 'outer' (union default) along axis = 1
# NOTE: the rows labels are repeated because is perfomming normal vertical stacking
# but the missing values are replaced with NaN to complete the union
outer_df = pd.concat([df1, df2], axis = 0, join = 'outer')

# vertical stacking along axis = 0
# and join = 'inner' (interception) along axis = 1 (columns)
inner_df = pd.concat([df1, df2], axis = 0, join = 'inner')

display_side_by_side(outer_df, inner_df)

A 
 B 
 C 
 D 
 
 
 
 
 0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 A1 
 B1 
 C1 
 D1 
 
 
 2 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 A3 
 B3 
 C3 
 D3 
 
 
 2 
 NaN 
 B4 
 C4 
 D4 
 
 
 3 
 NaN 
 B5 
 C5 
 D5 
 
 
 4 
 NaN 
 B6 
 C6 
 D6 
 
 
 5 
 NaN 
 B7 
 C7 
 D7 
 
 
 
 
 
 
 B 
 C 
 D 
 
 
 
 
 0 
 B0 
 C0 
 D0 
 
 
 1 
 B1 
 C1 
 D1 
 
 
 2 
 B2 
 C2 
 D2 
 
 
 3 
 B3 
 C3 
 D3 
 
 
 2 
 B4 
 C4 
 D4 
 
 
 3 
 B5 
 C5 
 D5 
 
 
 4 
 B6 
 C6 
 D6 
 
 
 5 
 B7 
 C7 
 D7

In [38]:
# horizontal stacking along axis = 1
# and join = 'outer' (union default) along axis = 0
outer_df = pd.concat([df1, df2], axis = 1, join = 'outer')

# horizontal stacking along axis = 1
# and join = 'inner' (interception) along axis = 0 (rows)
inner_df = pd.concat([df1, df2], axis = 1, join = 'inner')

display_side_by_side(outer_df, inner_df)

,A,B,C,D,B,C,D
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B4,C4,D4
3,A3,B3,C3,D3,B5,C5,D5
4,NaN,NaN,NaN,NaN,B6,C6,D6
5,NaN,NaN,NaN,NaN,B7,C7,D7
,A,B,C,D,B,C,D
2,A2,B2,C2,D2,B4,C4,D4
3,A3,B3,C3,D3,B5,C5,D5


### Concatenating with mixed dimns

It is also possible to concatenate combination of `Series` and/or `DataFrame`.

1. Appending columns can be done directly by concatenating directly the `DataFrame` and `Series`. e.g: `pd.concat([df1, s1], axis = 1)`.
2. However, for appending rows, it is neecesary to convert the series to `DataFrame` and transpose. e.g: `pd.concat([df1, s1.to_frame().T])`

**NOTE:** The name of a `Series` will be converted in the name of the column (or row) in the resulting
`DataFrame`. Also, you need to have a match on the `index` of the `Series` with the respective labels of the `DataFrame`.

**NOTE:** If `Series` has no name, a default name will be assigned. Also, if you concatenate only
series, you can rename the columns using the `keys` attribute.

In [40]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

In [39]:
# 1. Concatenating 1 or 2 Series as new columns (axis = 1) to a DataFrame
s1 = pd.Series( ["F0", "F1", "F2", "F3"], name="F")
s2 = pd.Series( ["G0", "G1", "G2", "G3"], name="G")

result1 = pd.concat([df1, s1], axis=1)
result2 = pd.concat([df1, s1, s2], axis=1)

display(df1)
display(result1)
display(result2)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


,A,B,C,D,F
0,A0,B0,C0,D0,F0
1,A1,B1,C1,D1,F1
2,A2,B2,C2,D2,F2
3,A3,B3,C3,D3,F3


,A,B,C,D,F,G
0,A0,B0,C0,D0,F0,G0
1,A1,B1,C1,D1,F1,G1
2,A2,B2,C2,D2,F2,G2
3,A3,B3,C3,D3,F3,G3


In [49]:
# 2. Concatenating 1 or 2 Series as new rows to a DataFrame
# NOTE: you need to match the index with the columns labels
s1 = pd.Series( ["A5", "B5", "C5", "D5"], index=["A","B","C","D"], name = 5)
s2 = pd.Series( ["A7", "B7", "C7", "D7"], index=["A","B","C","D"], name = 7)

result1 = pd.concat([df1, s1.to_frame().T])
result2 = pd.concat([df1, s1.to_frame().T, s2.to_frame().T])

display_side_by_side(df1, result1, result2)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
5,A5,B5,C5,D5


In [48]:
# Renaming a concatenation of series using keys
s1 = pd.Series( ["E0", "E1", "E2", "E3"], name="E")
s2 = pd.Series( ["F0", "F1", "F2", "F3"])
s3 = pd.Series( ["G0", "G1", "G2", "G3"])

result1 = pd.concat([s1, s2, s3], axis=1)
result2 = pd.concat([s1, s2, s3], axis=1, keys=["E","F","G"])
display_side_by_side(result1, result2)

,E,0,1
0,E0,F0,G0
1,E1,F1,G1
2,E2,F2,G2
3,E3,F3,G3
,E,F,G
0,E0,F0,G0
1,E1,F1,G1
2,E2,F2,G2
3,E3,F3,G3


## Database-style DataFrame (`join()` and `merge()`)
Pandas has full-featured, high performance in-memory join operations idiomatically very similar to relational databases like SQL.

Mainly, we will use `merge()` and we see later that `join()` is a specific case of `merge()`.

## Merging: `merge()`

Merging can performs different joining operations (like SQL): one-to-one, many-to-one,
many-to-many using the following function:

```
merge(
    left,
    right,
    how = 'inner',
    on = None,
    left_on = None,
    right_on = None,
    left_index = False,
    right_index = False
)
```

- `left` and `right` are two DataFrame or named Series objects.
- `how` define the index to keep in the result. It can be `left`, `right`, `outer`, `inner`, `cross`.
- `on` defines column or index level names to join on. Must be found in both the `left` and `right` objects. If not passed and `left_index` and `right_index` are False, the intersection of the columns in the two objects will be inferred to be the join keys.
- `left_on`: Columns or index levels from the left object. Can either be column names, index level names, or arrays with length equal to the length of the left object.
- `right_on`: Same usage as `left_on` for the right object.
- `left_index`: If True, use the index (row labels) from the left object as its join key(s). In the case of an object with a MultiIndex (hierarchical), the number of levels must match the number of join keys from the right object.
- `right_index`: Same usage as `left_index` for the right DataFrame or Series

**NOTE:** You can use `merge` as `pd.merge(df1, df2)` or `df1.merge(df2)`

In the following cells, we will practice the usage of these attributes, and how they are useful to perform (Database like) joining operations.

Things to take into account

- one to one, many to one, many to many (QUESTION many to many works in Pandas? Or how it woks)
- how 
- on, left_on, and right_on
- left_index, right_index, -> join

- mergin on columns and index using ???
- Extras: `indicator` argument


The related `join()` method, uses merge internally for the index-on-index (by default) and column(s)-on-index join.

In [57]:
# checking one to one merge()

left = pd.DataFrame(
    {
        "A": letter_list("A",4),
        "B": letter_list("B",4),
        "C": letter_list("C",4),
        "D": letter_list("D",4)
    }
)
right = pd.DataFrame(
    {
        "A": letter_list("A",4, start = 2),
        "F": letter_list("F",4, start = 2),
        "G": letter_list("G",4, start = 2),
        "H": letter_list("H",4, start = 2)
    }
)

display_side_by_side(left, right)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
,A,F,G,H
0,A2,F2,G2,H2
1,A3,F3,G3,H3
2,A4,F4,G4,H4
3,A5,F5,G5,H5


In [58]:
# one to one with how = "inner" (interception of left and right keys)
pd.merge(left, right, on="A", how="inner")

,A,B,C,D,F,G,H
0,A2,B2,C2,D2,F2,G2,H2
1,A3,B3,C3,D3,F3,G3,H3


In [59]:
# one to one with how = "outer" (union of left and right keys)
# NOTE: filling missing values with NaN
pd.merge(left, right, on="A", how="outer")

,A,B,C,D,F,G,H
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,F2,G2,H2
3,A3,B3,C3,D3,F3,G3,H3
4,A4,NaN,NaN,NaN,F4,G4,H4
5,A5,NaN,NaN,NaN,F5,G5,H5


In [60]:
# one to one with how = "left" (keep only left keys)
pd.merge(left, right, on="A", how="left")

,A,B,C,D,F,G,H
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,F2,G2,H2
3,A3,B3,C3,D3,F3,G3,H3


In [62]:
# one to one with how = "right" (keep only right keys)
pd.merge(left, right, on="A", how="right")

,A,B,C,D,F,G,H
0,A2,B2,C2,D2,F2,G2,H2
1,A3,B3,C3,D3,F3,G3,H3
2,A4,NaN,NaN,NaN,F4,G4,H4
3,A5,NaN,NaN,NaN,F5,G5,H5


In [65]:
# how = "cross" (cartesian product between rows in left and right)
# NOTE: it is not one to one, it is a cartesian product
pd.merge(left, right, how="cross")

,A_x,B,C,D,A_y,F,G,H
0,A0,B0,C0,D0,A2,F2,G2,H2
1,A0,B0,C0,D0,A3,F3,G3,H3
2,A0,B0,C0,D0,A4,F4,G4,H4
3,A0,B0,C0,D0,A5,F5,G5,H5
4,A1,B1,C1,D1,A2,F2,G2,H2
5,A1,B1,C1,D1,A3,F3,G3,H3
6,A1,B1,C1,D1,A4,F4,G4,H4
7,A1,B1,C1,D1,A5,F5,G5,H5
8,A2,B2,C2,D2,A2,F2,G2,H2
9,A2,B2,C2,D2,A3,F3,G3,H3


In [68]:
# checking one to many merge()

left = pd.DataFrame(
    {
        "A": letter_list("A",4),
        "B": letter_list("B",4),
        "C": letter_list("C",4),
        "D": letter_list("D",4)
    }
)
right = pd.DataFrame(
    {
        "A": ["A0", "A0", "A1", "A1", "A5"],
        "F": letter_list("F",5),
        "G": letter_list("G",5),
        "H": letter_list("H",5)
    }
)

display_side_by_side(left, right)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
,A,F,G,H
0,A0,F0,G0,H0
1,A0,F1,G1,H1
2,A1,F2,G2,H2
3,A1,F3,G3,H3
4,A5,F4,G4,H4


In [69]:
# one to one with how = "inner" (interception of left and right keys)
pd.merge(left, right, on="A", how="inner")

,A,B,C,D,F,G,H
0,A0,B0,C0,D0,F0,G0,H0
1,A0,B0,C0,D0,F1,G1,H1
2,A1,B1,C1,D1,F2,G2,H2
3,A1,B1,C1,D1,F3,G3,H3


In [ ]:
result = pd.merge(left, right, on=["key1", "key2"])
result

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [ ]:
result = pd.merge(left, right, on=["key1", "key2"], how="left")
display_side_by_side(left, result)

key1 
 key2 
 A 
 B 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 
 
 3 
 K2 
 K1 
 A3 
 B3 
 
 
 
 
 
 
 key1 
 key2 
 A 
 B 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 NaN 
 NaN 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 C1 
 D1 
 
 
 3 
 K1 
 K0 
 A2 
 B2 
 C2 
 D2 
 
 
 4 
 K2 
 K1 
 A3 
 B3 
 NaN 
 NaN

In [ ]:
result = pd.merge(left, right, on=["key1", "key2"], how="right")
display_side_by_side(right, result)

key1 
 key2 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 C0 
 D0 
 
 
 1 
 K1 
 K0 
 C1 
 D1 
 
 
 2 
 K1 
 K0 
 C2 
 D2 
 
 
 3 
 K2 
 K0 
 C3 
 D3 
 
 
 
 
 
 
 key1 
 key2 
 A 
 B 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 K1 
 K0 
 A2 
 B2 
 C1 
 D1 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 C2 
 D2 
 
 
 3 
 K2 
 K0 
 NaN 
 NaN 
 C3 
 D3

In [ ]:
result = pd.merge(left, right, on=["key1", "key2"], how="outer")
display_side_by_side(left, right, result)

key1 
 key2 
 A 
 B 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 
 
 3 
 K2 
 K1 
 A3 
 B3 
 
 
 
 
 
 
 key1 
 key2 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 C0 
 D0 
 
 
 1 
 K1 
 K0 
 C1 
 D1 
 
 
 2 
 K1 
 K0 
 C2 
 D2 
 
 
 3 
 K2 
 K0 
 C3 
 D3 
 
 
 
 
 
 
 key1 
 key2 
 A 
 B 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 C0 
 D0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 NaN 
 NaN 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 C1 
 D1 
 
 
 3 
 K1 
 K0 
 A2 
 B2 
 C2 
 D2 
 
 
 4 
 K2 
 K1 
 A3 
 B3 
 NaN 
 NaN 
 
 
 5 
 K2 
 K0 
 NaN 
 NaN 
 C3 
 D3

In [ ]:
result = pd.merge(left, right, how="cross")
display_side_by_side(left, right, result)

key1 
 key2 
 A 
 B 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 
 
 1 
 K0 
 K1 
 A1 
 B1 
 
 
 2 
 K1 
 K0 
 A2 
 B2 
 
 
 3 
 K2 
 K1 
 A3 
 B3 
 
 
 
 
 
 
 key1 
 key2 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 C0 
 D0 
 
 
 1 
 K1 
 K0 
 C1 
 D1 
 
 
 2 
 K1 
 K0 
 C2 
 D2 
 
 
 3 
 K2 
 K0 
 C3 
 D3 
 
 
 
 
 
 
 key1_x 
 key2_x 
 A 
 B 
 key1_y 
 key2_y 
 C 
 D 
 
 
 
 
 0 
 K0 
 K0 
 A0 
 B0 
 K0 
 K0 
 C0 
 D0 
 
 
 1 
 K0 
 K0 
 A0 
 B0 
 K1 
 K0 
 C1 
 D1 
 
 
 2 
 K0 
 K0 
 A0 
 B0 
 K1 
 K0 
 C2 
 D2 
 
 
 3 
 K0 
 K0 
 A0 
 B0 
 K2 
 K0 
 C3 
 D3 
 
 
 4 
 K0 
 K1 
 A1 
 B1 
 K0 
 K0 
 C0 
 D0 
 
 
 5 
 K0 
 K1 
 A1 
 B1 
 K1 
 K0 
 C1 
 D1 
 
 
 6 
 K0 
 K1 
 A1 
 B1 
 K1 
 K0 
 C2 
 D2 
 
 
 7 
 K0 
 K1 
 A1 
 B1 
 K2 
 K0 
 C3 
 D3 
 
 
 8 
 K1 
 K0 
 A2 
 B2 
 K0 
 K0 
 C0 
 D0 
 
 
 9 
 K1 
 K0 
 A2 
 B2 
 K1 
 K0 
 C1 
 D1 
 
 
 10 
 K1 
 K0 
 A2 
 B2 
 K1 
 K0 
 C2 
 D2 
 
 
 11 
 K1 
 K0 
 A2 
 B2 
 K2 
 K0 
 C3 
 D3 
 
 
 12 
 K2 
 K1 
 A3 
 B3 
 K0 
 K0 
 C0 
 D0 
 
 
 13 
 K2 
 K1 
 A3 
 B3 
 K1 
 K0 
 C1 
 D1 
 
 
 14 
 K2 
 K1 
 A3 
 B3 
 K1 
 K0 
 C2 
 D2 
 
 
 15 
 K2 
 K1 
 A3 
 B3 
 K2 
 K0 
 C3 
 D3

In [ ]:
ser = pd.Series(
    ["a", "b", "c", "d", "e", "f"],
    index=pd.MultiIndex.from_arrays(
        [["A", "B", "C"] * 2, [1, 2, 3, 4, 5, 6]], names=["Let", "Num"]
    ),
)
ser

Let  Num
A    1      a
B    2      b
C    3      c
A    4      d
B    5      e
C    6      f
dtype: object

In [ ]:
ser.reset_index()

,Let,Num,0
0,A,1,a
1,B,2,b
2,C,3,c
3,A,4,d
4,B,5,e
5,C,6,f


**Warning:** Joining / merging on duplicate keys can cause a returned frame that is the multiplication of the row dimensions, which may result in memory overflow. It is the user’ s responsibility to manage duplicate values in keys before joining large DataFrames.

In [ ]:
left = pd.DataFrame({"A": [1, 2], "B": [2, 2]})

right = pd.DataFrame({"A": [4, 5, 6], "B": [2, 2, 2]})

result = pd.merge(left, right, on="B", how="outer")

display_side_by_side(left, right, result)

# este ejercicio me hace dudar todo lo que entendi antes

A 
 B 
 
 
 
 
 0 
 1 
 2 
 
 
 1 
 2 
 2 
 
 
 
 
 
 
 A 
 B 
 
 
 
 
 0 
 4 
 2 
 
 
 1 
 5 
 2 
 
 
 2 
 6 
 2 
 
 
 
 
 
 
 A_x 
 B 
 A_y 
 
 
 
 
 0 
 1 
 2 
 4 
 
 
 1 
 1 
 2 
 5 
 
 
 2 
 1 
 2 
 6 
 
 
 3 
 2 
 2 
 4 
 
 
 4 
 2 
 2 
 5 
 
 
 5 
 2 
 2 
 6

In [ ]:

result = pd.merge(left, right, on="B", how="inner")

display_side_by_side(left, right, result)

A 
 B 
 
 
 
 
 0 
 1 
 2 
 
 
 1 
 2 
 2 
 
 
 
 
 
 
 A 
 B 
 
 
 
 
 0 
 4 
 2 
 
 
 1 
 5 
 2 
 
 
 2 
 6 
 2 
 
 
 
 
 
 
 A_x 
 B 
 A_y 
 
 
 
 
 0 
 1 
 2 
 4 
 
 
 1 
 1 
 2 
 5 
 
 
 2 
 1 
 2 
 6 
 
 
 3 
 2 
 2 
 4 
 
 
 4 
 2 
 2 
 5 
 
 
 5 
 2 
 2 
 6

If a string matches both a column name and an index level name, then a warning is issued and the column takes precedence. This will result in an ambiguity error in a future version.

**Note**: the non-unique keys problem that I didn't understand happens because
there are overlapping cases where for example

![image.png](attachment:image.png)

If we want to 'inner' merge using k as key we encounter overlapping in the values
of v because

1. left has K0 1
2. right has K0 3 and K0 4

Then, the overlapping is solve by adding two columns at the resultin merged df
as v_x and v_y

v_x for the left values of column v and v_y for the right values of column v

In [ ]:
df1 = pd.DataFrame({"v": [1, 2, 3]}, index=["K0", "K1", "K2"])
df2 = pd.DataFrame({"v": [4, 5, 6]}, index=["K0", "K0", "K3"])
df3 = pd.DataFrame({"v": [7, 8, 9]}, index=["K1", "K1", "K2"])

result = df1.join([df2, df3])

display_side_by_side(df1, df2, df3, result)


,v
K0,1
K1,2
K2,3
,v
K0,4
K0,5
K3,6
,v
K1,7
K1,8


In [ ]:
#the idea of combine_first is to have two data frames that are similar in the sense
# that they have same index and columns (not necessarily all of them)
# and you want to "patch" or fill some nan values from left with the values
# from right.

df1 = pd.DataFrame(
    [[np.nan, 3.0, 5.0], [-4.6, np.nan, np.nan], [np.nan, 7.0, np.nan]]
)


df2 = pd.DataFrame([[-42.6, np.nan, -8.2], [-5.0, 1.6, 4]], index=[1, 2])

result = df1.combine_first(df2)

display_side_by_side(df1, df2, result)

,0,1,2
0,NaN,3.0,5.0
1,-4.6,NaN,NaN
2,NaN,7.0,NaN
,0,1,2
1,-42.6,NaN,-8.2
2,-5.0,1.6,4.0
,0,1,2
0,NaN,3.0,5.0
1,-4.6,NaN,-8.2
2,-5.0,7.0,4.0


In [ ]:
# related method that alterate the NaN values in place
# but replace all the values from right into the left structure
df1.update(df2)
df1

,0,1,2
0,NaN,3.0,5.0
1,-42.6,NaN,-8.2
2,-5.0,1.6,4.0
